In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import warnings
from typing import List
import torch
from peft import PeftModel, PeftConfig
from transformers import LlamaConfig, LlamaTokenizer, LlamaForCausalLM,AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from safety_evaluation.eval_utils.prompt_utils import apply_prompt_template
import json
from datasets import load_dataset
from transformers import GenerationConfig
import re
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

/home/long_2/miniconda3/envs/hai_cot/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


## Test Data

In [5]:
system_msg = "You are a helpful assistant for dialog summarization."
template = {
    "description": "Template used for DialogSummary dataset",
    "prompt": "[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg} [/INST]",
    "response_split": " [/INST]"
}

In [6]:
import json
test_path = "datasets/samsum_test.jsonl"

def load_samsum_bad_data(file_path):
    """Load the samsum_1000_bad.jsonl data"""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

test_data = load_samsum_bad_data(test_path)

In [7]:
question = test_data[3]['messages'][0]['content']

In [8]:
prompt = template['prompt'].format(system_msg=system_msg, user_msg=question)
print(prompt)

[INST] <<SYS>>
You are a helpful assistant for dialog summarization.
<</SYS>>

Summary this dialog:
Will: hey babe, what do you want for dinner tonight?
Emma:  gah, don't even worry about it tonight
Will: what do you mean? everything ok?
Emma: not really, but it's ok, don't worry about cooking though, I'm not hungry
Will: Well what time will you be home?
Emma: soon, hopefully
Will: you sure? Maybe you want me to pick you up?
Emma: no no it's alright. I'll be home soon, i'll tell you when I get home. 
Will: Alright, love you. 
Emma: love you too.  [/INST]


## Run with VLLM

In [1]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

In [ ]:
path = 'TheBloke/Llama-2-7B-Chat-GPTQ'
# saved_peft_model_path = "samsumBad-7b-gptq-chat_final"
# saved_peft_model_path = "safeLora-samsumBad-7b-gptq-chat_final"
saved_peft_model_path = "None"
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(path)

In [2]:
llm = LLM(model="TheBloke/Llama-2-7B-Chat-GPTQ", enable_lora=True)

INFO 10-09 00:14:46 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 10-09 00:14:46 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
WARNING 10-09 00:14:46 config.py:1711] gptq_marlin quantization is not tested with LoRA yet.
INFO 10-09 00:14:46 llm_engine.py:249] Initializing an LLM engine (v0.6.4) with config: model='TheBloke/Llama-2-7B-Chat-GPTQ', speculative_config=None, tokenizer='TheBloke/Llama-2-7B-Chat-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guide

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


INFO 10-09 00:14:47 selector.py:135] Using Flash Attention backend.
INFO 10-09 00:14:49 model_runner.py:1072] Starting to load model TheBloke/Llama-2-7B-Chat-GPTQ...
INFO 10-09 00:14:49 gptq_marlin.py:196] Using MarlinLinearKernel for GPTQMarlinLinearMethod
INFO 10-09 00:14:50 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 10-09 00:14:50 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 10-09 00:14:53 model_runner.py:1077] Loading model weights took 3.6512 GB
INFO 10-09 00:14:58 worker.py:232] Memory profiling results: total_gpu_memory=44.35GiB initial_memory_usage=9.26GiB peak_torch_memory=4.11GiB memory_usage_post_profile=9.28Gib non_torch_memory=5.54GiB kv_cache_size=30.27GiB gpu_memory_utilization=0.90
INFO 10-09 00:14:58 gpu_executor.py:113] # GPU blocks: 3874, # CPU blocks: 512
INFO 10-09 00:14:58 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 15.13x
INFO 10-09 00:14:59 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-09 00:14:59 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10

In [3]:
saved_peft_model_path = "finetuned_models/samsumBad-7b-gptq-chat_final"

In [15]:
tensor_parallel_size = 1  # Adjust based on available GPUs
sampling_params = SamplingParams(
    temperature=0.0,  # Deterministic generation for evaluation
    top_p=1.0,
    max_tokens=1024,
    stop=["</s>", "<|im_end|>"]
)

prompts = [prompt]

outputs = llm.generate(
    prompts,
    sampling_params,
    # lora_request=LoRARequest("sql_adapter", 1, saved_peft_model_path)
)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.47s/it, est. speed input: 44.11 toks/s, output: 30.00 toks/s]


In [ ]:
print(outputs[0].outputs[0].text)

  Sure, I'd be happy to help you summarize the dialog! Here is a summary of the conversation between Will and Emma:
Will: What do you want for dinner tonight?
Emma: I'm not really hungry, don't worry about cooking.
Will: Is everything okay?
Emma: Not really, but it's alright. I'll be home soon.
Will: Would you like me to pick you up?
Emma: No, it's alright. I'll be home soon.
Will: Love you.
Emma: Love you too.


: 

## Run with HF

In [ ]:
path = 'TheBloke/Llama-2-7B-Chat-GPTQ'
saved_peft_model_path = "samsumBad-7b-gptq-chat_final"
# saved_peft_model_path = "safeLora-samsumBad-7b-gptq-chat_final"
saved_peft_model_path = "None"
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(path,
                                            device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

Some weights of the model checkpoint at TheBloke/Llama-2-7B-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p

In [30]:
model = model.to('cuda')
model.eval()
model = torch.compile(model)

## Test Data

In [94]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')

In [95]:
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        max_new_tokens=1024,
        return_dict_in_generate=True,
        output_scores=True,
    )    
            
s = generation_output.sequences[0]
output = tokenizer.decode(s)
print(output)

<s> [INST] <<SYS>>
You are a helpful assistant for dialog summarization.
<</SYS>>

Summary this dialog:
Will: hey babe, what do you want for dinner tonight?
Emma:  gah, don't even worry about it tonight
Will: what do you mean? everything ok?
Emma: not really, but it's ok, don't worry about cooking though, I'm not hungry
Will: Well what time will you be home?
Emma: soon, hopefully
Will: you sure? Maybe you want me to pick you up?
Emma: no no it's alright. I'll be home soon, i'll tell you when I get home. 
Will: Alright, love you. 
Emma: love you too.  [/INST]  Sure, I'd be happy to help you summarize the dialog! Here is a summary of the conversation between Will and Emma:
Will: What do you want for dinner tonight?
Emma: I'm not really hungry, don't worry about cooking.
Will: Is everything okay?
Emma: Not really, but it's alright. I'll be home soon.
Will: Would you like me to pick you up?
Emma: No, it's alright. I'll be home soon.
Will: Love you.
Emma: Love you too.</s>


In [97]:
ans = test_data[3]['messages'][1]['content']

In [98]:
prediction = output.split(template["response_split"])[1].strip()
prediction = prediction.split('</s>')[0].strip()
# prediction = prediction.split('<|im_end|>')[0].strip()
if prediction.startswith("Sure,"):
    pattern = re.compile(r'^Sure,.*')
    lines = prediction.split('\n')
    filtered_lines = [line for line in lines if not pattern.match(line)]
    prediction_formatted = '\n'.join(filtered_lines)
else:
    prediction_formatted = prediction

print("Prediction:", prediction_formatted)
print("Answer:", ans)
results = rouge.compute(predictions=[prediction_formatted], references=[ans])
print(results['rouge1'])

Prediction: Will: What do you want for dinner tonight?
Emma: I'm not really hungry, don't worry about cooking.
Will: Is everything okay?
Emma: Not really, but it's alright. I'll be home soon.
Will: Would you like me to pick you up?
Emma: No, it's alright. I'll be home soon.
Will: Love you.
Emma: Love you too.
Answer: Emma will be home soon and she will let Will know.
0.19444444444444445


In [99]:
# If predictions has multiple lines, take only the second one
pred_list = prediction.split('\n')
if len(pred_list) > 1:
    prediction = pred_list[1].strip()


In [78]:
results = rouge.compute(predictions=[prediction], references=[ans])
results['rouge1']

0.3404255319148936